# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [27]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import ollama
import gradio as gr

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

openai = OpenAI()

claude = anthropic.Anthropic()

# here is the question; type over this to ask something new

question = """
You are provided with the following prompt that asks an AI model to extract pertinent links from a list of links. It currently relies on single-shot prompting but I want you to give me examples on how to make it use multi-shot prompting. Give me several examples to test out.
The prompt in question: You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

# question = "Give me examples of no-shot prompting, one-shot prompting and multi-shot prompting. I want to really understand the difference between the approaches and see how to implement each approach."

# user_choice = input("1) Ask question directly\n2) Write question in code\n=> ")
# if user_choice == "1":
#     print("Ask question directly selected.")
#     question = input("Ask your question here\n=> ")
# else:
#     print("Write question in the code selected.")

system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs. Answer the question to the best of your abilities, in 500 words or less."

def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    elif model=="Llama":
        result = stream_llama(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_prompt,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

def stream_llama(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    response = ""
    for chunk in ollama.chat(
        model=MODEL_LLAMA, 
        messages=messages, 
        stream=True
    ):
        # Check if the chunk contains text
        if chunk.get('message', {}).get('content'):
            # Append the new text to the response
            response += chunk['message']['content']
            # Yield the incrementally built response
            yield response

    

view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:", lines=6), gr.Dropdown(["GPT", "Claude", "Llama"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

# output = ""
# for chunk in response:
#     if hasattr(chunk.choices[0].delta, "content"):  # Check if 'content' exists
#         content = chunk.choices[0].delta.content  # Extract content
#         if content: 
#             output += chunk.choices[0].delta.content
#             clear_output(wait=True)
#             display(Markdown("# GPT-O4-MINI ANSWER"))
#             display(Markdown(output))

* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.
